<a href="https://colab.research.google.com/github/JBenedictM/501-A4-TF/blob/master/501_A4_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


# Upload to google drive: notMNIST.npz

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/notMNIST.npz"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np

print("--Get data--")
# path is defined by the cell above
with np.load(path, allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

print("--Process data--")
print(len(y_train))
x_train, x_test = x_train / 255.0, x_test / 255.0

#shape = (batch_size, height, width, depth)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
 
 # new model employes more convolution layers 
 # each layer also contains more kernels
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(256, input_shape=(28, 28, 1), kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"),
  tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  #tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"),
  #tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("--Fit model--")
val_loss_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=20)
checkpoint_val = tf.keras.callbacks.ModelCheckpoint("best_notmnist_model.h5", monitor="val_loss", mode="min", save_best_only=True, verbose=1)
checkpoint_train = tf.keras.callbacks.ModelCheckpoint("best_notmnist_model.h5", monitor="loss", mode="min", save_best_only=True, verbose=1)

cb = [val_loss_callback, checkpoint_val, checkpoint_train]

model.fit(x_train, y_train, epochs=99999999,  verbose=1, validation_data=(x_test, y_test), callbacks=cb)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(x_test,  y_test, verbose=2)
model.summary()
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

--Get data--
--Process data--
60000
--Make model--
--Fit model--
Train on 60000 samples, validate on 10000 samples
Epoch 1/99999999
59936/60000 [============================>.] - ETA: 0s - loss: 2.0345 - accuracy: 0.2442
Epoch 00001: val_loss improved from inf to 0.72238, saving model to best_notmnist_model.h5

Epoch 00001: loss improved from inf to 2.03355, saving model to best_notmnist_model.h5
60000/60000 [==============================] - 45s 758us/sample - loss: 2.0336 - accuracy: 0.2445 - val_loss: 0.7224 - val_accuracy: 0.7735
Epoch 2/99999999
59904/60000 [============================>.] - ETA: 0s - loss: 0.8021 - accuracy: 0.7560
Epoch 00002: val_loss improved from 0.72238 to 0.30524, saving model to best_notmnist_model.h5

Epoch 00002: loss improved from 2.03355 to 0.80194, saving model to best_notmnist_model.h5
60000/60000 [==============================] - 41s 678us/sample - loss: 0.8019 - accuracy: 0.7560 - val_loss: 0.3052 - val_accuracy: 0.9090
Epoch 3/99999999
59936/6000

In [0]:
#Save Model to notMNIST.h5

In [0]:
from google.colab import files
files.download('best_notmnist_model.h5')